In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad']]
selected_features.head()

,koi_disposition,koi_period,koi_time0bk,koi_slogg,koi_srad
0,CONFIRMED,54.418383,162.513840,4.467,0.927
1,FALSE POSITIVE,19.899140,175.850252,4.544,0.868
2,FALSE POSITIVE,1.736952,170.307565,4.564,0.791
3,CONFIRMED,2.525592,171.595550,4.438,1.046
4,CONFIRMED,4.134435,172.979370,4.486,0.972


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
X = selected_features.drop("koi_disposition", axis=1)
y = selected_features["koi_disposition"]
print(X.shape, y.shape)

(6991, 4) (6991,)


In [7]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
X_train.head()

,koi_period,koi_time0bk,koi_slogg,koi_srad
3563,10.548413,139.064020,4.387,1.092
4099,24.754385,140.207320,4.519,0.804
5460,1.057336,131.792007,4.594,0.683
1091,201.118319,187.569860,4.447,0.954
5999,91.649983,175.715600,4.145,1.608


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [11]:
from tensorflow.keras.utils import to_categorical

# One-hot encoding
# YOUR CODE HERE
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [33]:
# First, create a normal neural network with 4 inputs, 6 hidden nodes, and 3 outputs
# YOUR CODE HERE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=4))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 6)                 30        
                                                                 
 dense_14 (Dense)            (None, 8)                 56        
                                                                 
 dense_15 (Dense)            (None, 3)                 27        
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
# YOUR CODE HERE
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
# Fit the model to the training data
# YOUR CODE HERE
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 0s - loss: 1.0911 - accuracy: 0.4082 - 374ms/epoch - 2ms/step
Epoch 2/100
164/164 - 0s - loss: 1.0440 - accuracy: 0.4949 - 142ms/epoch - 866us/step
Epoch 3/100
164/164 - 0s - loss: 1.0424 - accuracy: 0.4949 - 142ms/epoch - 863us/step
Epoch 4/100
164/164 - 0s - loss: 1.0410 - accuracy: 0.4949 - 137ms/epoch - 833us/step
Epoch 5/100
164/164 - 0s - loss: 1.0396 - accuracy: 0.4949 - 138ms/epoch - 843us/step
Epoch 6/100
164/164 - 0s - loss: 1.0378 - accuracy: 0.4949 - 142ms/epoch - 867us/step
Epoch 7/100
164/164 - 0s - loss: 1.0359 - accuracy: 0.4949 - 134ms/epoch - 817us/step
Epoch 8/100
164/164 - 0s - loss: 1.0336 - accuracy: 0.4949 - 132ms/epoch - 806us/step
Epoch 9/100
164/164 - 0s - loss: 1.0313 - accuracy: 0.4949 - 149ms/epoch - 908us/step
Epoch 10/100
164/164 - 0s - loss: 1.0283 - accuracy: 0.4949 - 126ms/epoch - 768us/step
Epoch 11/100
164/164 - 0s - loss: 1.0261 - accuracy: 0.4949 - 125ms/epoch - 760us/step
Epoch 12/100
164/164 - 0s - loss: 1.0234 - accuracy: 0

In [36]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.9561 - accuracy: 0.5406 - 208ms/epoch - 4ms/step
Normal Neural Network - Loss: 0.9560554623603821, Accuracy: 0.540617823600769


# Train the Model



In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [45]:
# Create the GridSearchCV model
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC(gamma='auto')
clf = GridSearchCV(svc, parameters)

# Train the model with GridSearch
clf.fit(X_test_scaled, encoded_y_test)
df = pd.DataFrame(clf.cv_results_)
print(clf.best_params_)
print(clf.best_score_)

{'C': 1, 'kernel': 'linear'}
0.5200229226361032


In [44]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,1,rbf,0.980000
2,10,linear,0.973333
3,10,rbf,0.980000


In [53]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [54]:
model_params = {
    'svm' : {
        'model' : svm.SVC(gamma='auto'),
        'params' : {
            'C' : [1,5,10],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
    'model' : LogisticRegression(solver='liblinear',multi_class='auto'),
    'params' : { 'C': [1,5,10]
    }
  }
}

In [55]:
scores= []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_test_scaled, encoded_y_test)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_})

In [58]:
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.520023,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.519468,{'n_estimators': 10}
2,logistic_regression,0.520023,{'C': 1}


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'antoinetteboyle.sav'
joblib.dump(model, filename)